<a href="https://colab.research.google.com/github/sunamkundal01/machine-learning-tutorials/blob/main/fever_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Collect data from chatgpt and store it in excel
##Text: "I have been feeling hot and shivering at night."
##Label: 1 (fever)
##Text: "I have a sore throat but no temperature."
##Label: 0 (no fever)

In [1]:
from google.colab import files
uploaded = files.upload()



Saving fever_detection_dataset.xlsx to fever_detection_dataset.xlsx


In [ ]:
import pandas as pd
df = pd.read_csv('fever_detection_dataset.xlsx')


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import string
import nltk
from nltk.corpus import stopwords


In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
df = pd.read_excel('fever_detection_dataset.xlsx')

In [6]:
df.head()

,Symptom Description,Fever Indication
0,I have chills and feel light-headed frequently.,1
1,"I have a mild headache, but my energy levels a...",0
2,"My body feels weak, and I have been sweating p...",1
3,"I feel a bit light-headed, but my temperature ...",0
4,"My body feels weak, and I have been sweating p...",1


# Data Preprocessing

In [5]:

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [7]:
df['Processed Text'] = df['Symptom Description'].apply(preprocess_text)

In [18]:
df.head()


,Symptom Description,Fever Indication,Processed Text
0,I have chills and feel light-headed frequently.,1,chills feel lightheaded frequently
1,"I have a mild headache, but my energy levels a...",0,mild headache energy levels normal
2,"My body feels weak, and I have been sweating p...",1,body feels weak sweating profusely
3,"I feel a bit light-headed, but my temperature ...",0,feel bit lightheaded temperature feels fine
4,"My body feels weak, and I have been sweating p...",1,body feels weak sweating profusely


In [19]:
df.size

1500

#Vectorization with TF-IDF

In [10]:
vectorizer = TfidfVectorizer(max_df=0.8, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['Processed Text'])
y = df['Fever Indication']


# Split data

In [11]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Model Training with Naive Bayes

In [13]:
# Initialize the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train, y_train)


MultinomialNB()

In [15]:
# Make predictions on the test set
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # Import the confusion_matrix function
y_pred = nb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        47
           1       1.00      1.00      1.00        53

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

Confusion Matrix:
 [[47  0]
 [ 0 53]]


# Now  predict whether a given review indicates fever or not

In [20]:

def predict_fever(review):
    # Preprocess the review text
    processed_review = preprocess_text(review)

    # Vectorize the review
    review_vectorized = vectorizer.transform([processed_review])

    # Predict using the trained Naive Bayes model
    prediction = nb_model.predict(review_vectorized)

    # Display the result
    if prediction == 1:
        return "Fever"
    else:
        return "No Fever"

# Example of how to get the prediction
new_review = input("Enter the patient's symptom description: ")
prediction_result = predict_fever(new_review)
print(f"Prediction: {prediction_result}")










Enter the patient's symptom description: I have been feeling feverish, shivering, and sweating at night. Prediction: Fever
Prediction: Fever


In [21]:
prediction_result


'Fever'

In [22]:
res=predict_fever('I have a sore throat but no other symptoms.')

In [23]:
res

'No Fever'

hence our model perdicts the fever based on the patient review .
